In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import StratifiedKFold, train_test_split

import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras import regularizers as R
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras import layers as L
from tensorflow.keras import optimizers as O
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy
tf.keras.utils.set_random_seed(722)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv', index_col='Id')
test_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv', index_col='Id')

train_df['EJ'] = train_df['EJ'].replace({'A': 0, 'B': 1})
test_df['EJ'] = test_df['EJ'].replace({'A': 0, 'B': 1})

nan_fill = train_df.isna().any()
nan_fill *= train_df.min() - train_df.max()
nan_fill[nan_fill == 0] = train_df.median()
train_df = train_df.fillna(nan_fill)
test_df = test_df.fillna(nan_fill)

In [3]:
X_test = test_df.values

In [4]:
# @tf.keras.utils.register_keras_serializable() => registers an object with the Keras serialization framework.
#        This allows you to use custom layer/class/function in your Keras models and ensure that it can be -
#        - serialized and deserialized correctly when saving and loading models.
#        This decorator injects the decorated class or function into the Keras custom object dictionary, so that -
#        - it can be serialized and deserialized without needing an entry in the user-provided custom object dict.


@tf.keras.utils.register_keras_serializable()
def smish(x):
    # smish(x)=x⋅tanh(ln(1+σ(x)))
    return x * K.tanh(K.log(1 + K.sigmoid(x)))


@tf.keras.utils.register_keras_serializable()
class GatedLinearUnit(Model): # L.Layer # Model my_
    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)
        self.linear = L.Dense(units)
        self.sigmoid = L.Dense(units, activation="sigmoid")
        self.units = units

    def get_config(self):
        config = super().get_config()
        config['units'] = self.units
        return config
    
    def call(self, inputs):
        return self.linear(inputs) * self.sigmoid(inputs)

    def model(self):
        # input_shape=(not include batch size).
        x = tf.keras.Input(shape=(self.units), batch_size=batch_size)
        return tf.keras.Model(inputs=x, outputs=self.call(x)) 
    

@tf.keras.utils.register_keras_serializable()
class GatedResidualNetwork(Model): # L.Layer # Model my_
    def __init__(self, units, dropout_rate, num_features, concat, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.dropout_rate = dropout_rate
        self.relu_dense = L.Dense(units, activation=smish)
        self.linear_dense = L.Dense(units)
        self.dropout = L.Dropout(dropout_rate)
        self.gated_linear_unit = GatedLinearUnit(units)
        self.layer_norm = L.LayerNormalization()
        self.project = L.Dense(units)
        self.num_features = num_features
        self.concat = concat

    def get_config(self):
        config = super().get_config()
        config['units'] = self.units
        config['dropout_rate'] = self.dropout_rate
        return config
    
    def call(self, inputs):
        x = self.relu_dense(inputs)
        x = self.linear_dense(x)
        x = self.dropout(x)
        if inputs.shape[-1] != self.units:
            inputs = self.project(inputs)
        x = inputs + self.gated_linear_unit(x)
        x = self.layer_norm(x)
        return x

    def model(self):
        # input_shape=(not include batch size).
        if (self.concat and 2*self.units-self.num_features): sh = self.num_features*(2*self.units-self.num_features)
        elif (self.concat and 2*self.units-self.num_features==0):  sh = self.num_features
        elif (self.concat==False and 2*self.units-self.num_features): sh = 2*self.units-self.num_features
        else: sh = 1        
        x = tf.keras.Input(shape=(sh), batch_size=batch_size)
        return tf.keras.Model(inputs=x, outputs=self.call(x)) 
    

@tf.keras.utils.register_keras_serializable()
class VariableSelection(Model): # L.Layer # Model my_
    def __init__(self, num_features, units, dropout_rate, **kwargs):
        super().__init__(**kwargs)

        # Create a GRN for the concatenation of all the features
        self.grn_concat = GatedResidualNetwork(units, dropout_rate, num_features, True)

        self.grns = list()
        # Create a GRN for each feature independently
        for idx in range(num_features):
            grn = GatedResidualNetwork(units, dropout_rate, num_features, False)
            self.grns.append(grn)
            
        self.softmax = L.Dense(units=num_features, activation="softmax")
        self.num_features = num_features
        self.units = units
        self.dropout_rate = dropout_rate

    def get_config(self):
        config = super().get_config()
        config['num_features'] = self.num_features
        config['units'] = self.units
        config['dropout_rate'] = self.dropout_rate
        return config
    
    def call(self, inputs):
        v = L.concatenate(inputs)
        v = self.grn_concat(v)
        v = tf.expand_dims(self.softmax(v), axis=-1)

        x = []
        for idx, input_ in enumerate(inputs):
            x.append(self.grns[idx](input_))
        x = tf.stack(x, axis=1)

        outputs = tf.squeeze(tf.matmul(v, x, transpose_a=True), axis=1)
        return outputs
        
    def model(self):
        sh = [2*self.units-self.num_features if 2*self.units-self.num_features else 1][0]
        x = []
        for idx in range(self.num_features):
            x.append(tf.keras.layers.Input(shape=(sh)))
        return tf.keras.Model(inputs=x, outputs=self.call(x))     

@tf.keras.utils.register_keras_serializable()
class VariableSelectionFlow(Model): # L.Layer # Model my_
    def __init__(self, num_features, units, dropout_rate, dense_units=None, **kwargs):
        super().__init__(**kwargs)
        self.variableselection = VariableSelection(num_features, units, dropout_rate)
        self.split = L.Lambda(lambda t: tf.split(t, num_features, axis=-1))
        self.dense = dense_units
        if dense_units:
            self.dense_list = [L.Dense(dense_units, \
                                       activation='linear') \
                               for _ in tf.range(num_features)
                              ]
        self.num_features = num_features
        self.units = units
        self.dropout_rate = dropout_rate
        self.dense_units = dense_units
        
    def get_config(self):
        config = super().get_config()
        config['num_features'] = self.num_features
        config['units'] = self.units
        config['dropout_rate'] = self.dropout_rate
        config['dense_units'] = self.dense_units
        return config        
    
    def call(self, inputs):   
        split_input = self.split(inputs)
        if self.dense:
            l = [self.dense_list[i](split_input[i]) for i in range(len(self.dense_list))]
        else:
            l = split_input
        return self.variableselection(l)           

    def model(self):
        # input_shape=(not include batch size).
        sh = 2*self.units-[self.dense_units if self.dense_units else 0][0]
        x = tf.keras.Input(shape=(sh), batch_size=batch_size)
        return tf.keras.Model(inputs=x, outputs=self.call(x)) 

In [5]:
mfolder = '/kaggle/input/icr-iarc-weights/icr-weights/'
models_weights = os.listdir(mfolder)
bypass = ['mod_f4_r0_tr0.1738_val0.2523.h5', 'mod_f7_r0_tr0.1936_val0.2573.h5']
#bypass = []
print(models_weights[:2])

['mod_f8_r1_tr0.1803_val0.1829.h5', 'mod_f0_r0_tr0.1455_val0.1582.h5']


In [6]:
y_pred = np.zeros_like(test_df.iloc[:,0].values, dtype=np.float32)
batch_size = 32

units_1 = 32
drop_1 = 0.75
dense_units = 8

units_2 = 16
drop_2 = 0.5

units_3 = 8
drop_3 = 0.25

for n, model_weights in enumerate(models_weights):
    if model_weights in bypass: continue
    inputs_1 = tf.keras.Input(shape=(56,))
        
    features_1 = VariableSelectionFlow(56, units_1, drop_1, dense_units=dense_units)(inputs_1)
    features_2 = VariableSelectionFlow(units_1, units_2, drop_2)(features_1)         
    features_3 = VariableSelectionFlow(units_2, units_3, drop_3)(features_2)         

    outputs = L.Dense(1, activation="sigmoid")(features_3)

    model = Model(inputs=inputs_1, outputs=outputs)      
    model.load_weights(mfolder + model_weights)
    y_pred += model.predict(X_test)[:,0]


1/1 [==============================] - 9s 9s/step


In [7]:
y_pred /= len(models_weights)
p1 = y_pred
p0 = 1 - p1
# p1.shape => (5,)
p = np.stack([p0, p1]).T
# p.shape => (5, 2)

class_0_est_instances = p[:,0].sum()
others_est_instances = p[:,1:].sum()

# reweighting the probabilities.
# np.array([[ 1/(class_0_est_instances if i==0 else others_est_instances) for i in range(p.shape[1]) ]]).shape => (1, 2)
new_p = p * np.array([[ 1/(class_0_est_instances if i==0 else others_est_instances) for i in range(p.shape[1]) ]])
# new_p.shape => (5, 2)
# np.sum(new_p, axis=1, keepdims=1).shape => (5, 1)
new_p = new_p / np.sum(new_p, axis=1, keepdims=1)

test_df[['class_0', 'class_1']] = new_p
test_df[['class_0', 'class_1']].to_csv('submission.csv')